In [1]:
import torch
import random
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import argparse,os,time
import os
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
num_gpus=4

## nn.CrossEntropyLoss VS nn.NLLLoss

nn.CrossEntropyLoss는 nn.LogSoftmax와 nn.NLLLoss의 연산의 조합이다. nn.LogSoftmax는 신경망 말단의 결과 값들을 확률개념으로 해석하기 위한 Softmax 함수의 결과에 log 값을 취한 연산이고, nn.NLLLoss는 nn.LogSoftmax의 log 결과값에 대한 교차 엔트로피 손실 연산(Cross Entropy Loss|Error)이다.

In [2]:
data=pd.read_csv("./Otto_Group_Product_data/train.csv")

In [3]:
data

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61873,61874,1,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,2,0,Class_9
61874,61875,4,0,0,0,0,0,0,0,0,...,0,2,0,0,2,0,0,1,0,Class_9
61875,61876,0,0,0,0,0,0,0,3,1,...,0,3,1,0,0,0,0,0,0,Class_9
61876,61877,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,3,10,0,Class_9


In [4]:
for i in range(10):
    data=data.replace("Class_{}".format(i),i)

In [5]:
shuffle_data=data.loc[:,"feat_1":].values

In [6]:
np.take(shuffle_data,np.random.permutation(shuffle_data.shape[0]),
        axis=0,out=shuffle_data)

array([[ 0,  0,  0, ...,  1,  0,  5],
       [ 0,  0, 23, ...,  0,  0,  6],
       [ 0,  0, 11, ...,  0,  0,  6],
       ...,
       [ 0,  0,  1, ...,  1,  1,  6],
       [ 0,  0,  0, ...,  1,  0,  5],
       [ 0,  0,  0, ...,  0,  0,  3]])

In [7]:
x_data=shuffle_data[:,:-1]
y_data=shuffle_data[:,-1]

In [8]:
mean_x=np.mean(x_data, axis=0)
std_x=np.std(x_data, axis=0)
x_train=(x_data - mean_x) / std_x

In [9]:
class CustomDataset(Dataset):
    def __init__(self,x_dat,y_dat):
        x = x_dat
        y = y_dat
        self.len = x.shape[0]
        y=y.astype('int')
        x=x.astype('float32')
        self.x_data = torch.tensor(x)
        self.y_data = torch.tensor(y)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [10]:
train_data_x, val_data_x, train_data_label, val_data_label = train_test_split(x_data, y_data, 
                                                                        test_size=0.2)

In [11]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(93, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 9),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        x=x.view(batch_size//num_gpus,-1)
        out=self.fc(x)
        return out

In [12]:
batch_size=256
train_dataset = CustomDataset(train_data_x,train_data_label)
train_loader = DataLoader(dataset=train_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)
val_dataset = CustomDataset(val_data_x,val_data_label)
val_loader = DataLoader(dataset=val_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)

In [13]:
model=nn.DataParallel(Model().cuda())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0.0001)

In [14]:
trn_loss_list = []
val_loss_list = []
total_epoch=100
model_char="minloss"
model_name=""
patience=5
start_early_stop_check=0
saving_start_epoch=10

for epoch in range(total_epoch):
    trn_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del output
    with torch.no_grad():
        val_loss = 0.0
        cor_match = 0
        for j, val in enumerate(val_loader):
            val_x, val_label = val
            if torch.cuda.is_available():
                val_x = val_x.cuda()
                val_label =val_label.cuda()
            val_output = model(val_x)
            v_loss = criterion(val_output, val_label)
            val_loss += v_loss
            _, predicted=torch.max(val_output,1)
            cor_match+=np.count_nonzero(predicted.cpu().detach()==val_label.cpu().detach())
    del val_output
    del v_loss
    del predicted
    
    
    
    trn_loss_list.append(trn_loss/len(train_loader))
    val_loss_list.append(val_loss/len(val_loader))
    val_acc=cor_match/(len(val_loader)*batch_size)
    now = time.localtime()
    print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))

    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | val accuracy: {:.4f}% \n".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), val_loss / len(val_loader), val_acc*100
            ))
    
    
    if epoch+1>2:
        if val_loss_list[-1]>val_loss_list[-2]:
            start_early_stop_check=1
    else:
        val_loss_min=val_loss_list[-1]
        
    if start_early_stop_check:
        early_stop_temp=val_loss_list[-patience:]
        if all(early_stop_temp[i]<early_stop_temp[i+1] for i in range (len(early_stop_temp)-1)):
            print("Early stop!")
            break
            
    if epoch+1>saving_start_epoch:
        if val_loss_list[-1]<val_loss_min:
            if os.path.isfile(model_name):
                os.remove(model_name)
            val_loss_min=val_loss_list[-1]
            model_name="Otto_model_"+model_char+"_{:.3f}".format(val_loss_min)
            torch.save(model, model_name)
            print("Model replaced and saved as ",model_name)

2020/09/28 12:05:22
epoch: 1/100 | trn loss: 1.3650 | val loss: 1.0495 | val accuracy: 66.0156% 

2020/09/28 12:05:35
epoch: 2/100 | trn loss: 4.3664 | val loss: 1.0405 | val accuracy: 62.9720% 

2020/09/28 12:05:48
epoch: 3/100 | trn loss: 0.9778 | val loss: 0.9105 | val accuracy: 68.7012% 

2020/09/28 12:06:02
epoch: 4/100 | trn loss: 0.9014 | val loss: 0.8667 | val accuracy: 70.1090% 

2020/09/28 12:06:15
epoch: 5/100 | trn loss: 0.8575 | val loss: 0.8500 | val accuracy: 70.3532% 

2020/09/28 12:06:28
epoch: 6/100 | trn loss: 0.8195 | val loss: 0.8213 | val accuracy: 71.3867% 

2020/09/28 12:06:42
epoch: 7/100 | trn loss: 0.7980 | val loss: 0.8154 | val accuracy: 71.0449% 

2020/09/28 12:06:55
epoch: 8/100 | trn loss: 0.7822 | val loss: 0.7975 | val accuracy: 70.8333% 

2020/09/28 12:07:08
epoch: 9/100 | trn loss: 0.7696 | val loss: 0.7395 | val accuracy: 72.2738% 

2020/09/28 12:07:21
epoch: 10/100 | trn loss: 0.7406 | val loss: 0.7962 | val accuracy: 72.7783% 

2020/09/28 12:07:34

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


2020/09/28 12:07:47
epoch: 12/100 | trn loss: 0.7184 | val loss: 0.7913 | val accuracy: 72.4609% 

2020/09/28 12:08:00
epoch: 13/100 | trn loss: 0.7056 | val loss: 0.7703 | val accuracy: 73.1038% 

Model replaced and saved as  Otto_model_minloss_0.770
2020/09/28 12:08:13
epoch: 14/100 | trn loss: 0.7024 | val loss: 0.7655 | val accuracy: 73.1445% 

Model replaced and saved as  Otto_model_minloss_0.766
2020/09/28 12:08:26
epoch: 15/100 | trn loss: 0.6831 | val loss: 0.7714 | val accuracy: 73.2829% 

2020/09/28 12:08:39
epoch: 16/100 | trn loss: 0.6792 | val loss: 0.7694 | val accuracy: 74.0397% 

2020/09/28 12:08:52
epoch: 17/100 | trn loss: 0.6699 | val loss: 0.7878 | val accuracy: 73.1852% 

2020/09/28 12:09:05
epoch: 18/100 | trn loss: 0.6734 | val loss: 0.7682 | val accuracy: 73.7549% 

2020/09/28 12:09:18
epoch: 19/100 | trn loss: 0.6602 | val loss: 0.7813 | val accuracy: 74.2188% 

2020/09/28 12:09:30
epoch: 20/100 | trn loss: 0.6484 | val loss: 0.7539 | val accuracy: 74.3490% 

M

2020/09/28 12:25:25
epoch: 93/100 | trn loss: 13.3061 | val loss: 14.7904 | val accuracy: 35.3027% 

2020/09/28 12:25:38
epoch: 94/100 | trn loss: 12.1539 | val loss: 7.3465 | val accuracy: 53.9714% 

2020/09/28 12:25:51
epoch: 95/100 | trn loss: 10.5777 | val loss: 7.7308 | val accuracy: 45.5973% 

2020/09/28 12:26:04
epoch: 96/100 | trn loss: 8.9386 | val loss: 8.3800 | val accuracy: 42.8955% 

2020/09/28 12:26:17
epoch: 97/100 | trn loss: 9.6195 | val loss: 7.0349 | val accuracy: 44.9707% 

2020/09/28 12:26:30
epoch: 98/100 | trn loss: 7.7411 | val loss: 7.7491 | val accuracy: 50.5697% 

2020/09/28 12:26:43
epoch: 99/100 | trn loss: 7.3574 | val loss: 7.2358 | val accuracy: 56.7546% 

2020/09/28 12:26:56
epoch: 100/100 | trn loss: 6.3696 | val loss: 3.9917 | val accuracy: 56.4941% 

